In [63]:
# Importo librerías

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from collections import Counter

In [64]:
# Obtengo credenciales

client_credentials_manager = SpotifyClientCredentials("bcd05cdd915c49429590886e25342140", 
                                                      "0eae1090f5c347769d17b38d4892e303")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [65]:
# De la aplicación voy a recibir una lista de nombres de usuarios
# Tomo la siguiente lista como ejemplo:

users = ["sedepa", "zltm3cqxt0u0s9k4e8oy564dk"]

In [66]:
# Para el paso 3 del layout de la app, voy a tener que devolver a cada usuario una lista con 
# el nombre de sus playlists, para que seleccione cuáles va a querer subir para crear la lista 
# conjunta.

# Por tanto, de la siguiente función voy a obtener un diccionario para devolver a la app, con
# el siguiente formato:

diccionario1 = {"user1": 
                   ["playlist1_name", "playlist2_name"],
                "user2": 
                   ["playlist1_name", "playlist2_name"]}


# Además, aunque al usuario en la app no se lo voy a mostrar, también voy a aprovechar para 
# obtener aquí el playlist_ID correspondiente a cada playlist, guardándolos en otro diccionario 
# con el siguiente formato:


diccionario2 = {"user1": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"},
                "user2": 
                   {"playlist1_name": "playlist1_ID", "playlist2_name": "playlist2_ID"}}

In [67]:
# Defino la función que obtenga ambos diccionarios a partir de una lista de usuarios.

def get_user_playlists(users):
    
    users_playlists_names = {}
    users_playlists_names_IDs = {}
    
    for user in users:
        playlists_names = []
        users_playlists_names_IDs[user] = {}
        playlists = sp.user_playlists(user)
        
        for i, playlist in enumerate(playlists["items"]):
            playlists_names.append(playlist["name"])
            users_playlists_names_IDs[user][playlist["name"]] = playlist['uri'][17:]
            
        users_playlists_names[user] = playlists_names
        
    return users_playlists_names, users_playlists_names_IDs


In [68]:
# Guardo cada diccionario en una variable

users_playlists_names = get_user_playlists(users)[0]
users_playlists_names_IDs = get_user_playlists(users)[1]

In [69]:
# El diccionario users_playlists_names va a ser devuelto a la app para que cada usuario
# seleccione de entre sus playlists cuáles quiere subir.

# Además, cada usuario va a poder filtrar las canciones de sus playlists seleccionadas 
# por los géneros que escoja.

# También van a tener que indicar cuál de ellos será el "host" (el usuario en cuyo perfil 
# se creará la lista conjunta), y el nombre de la lista conjunta. 

In [70]:
# Después de que los usuarios hayan seleccionado las playlists que quieren subir, el género
# de las canciones, el host y el nombre de la nueva lista, tengo que recibir 
# un diccionario con el siguiente formato:

mygroup = {"users": 
                   {"user1": {"playlists": [], "genres": []}, 
                    "user2": {"playlists": [], "genres": []}}, 
           "host": "user1",
           "new_playlist_name": "name"}

# Es posible que las listas de valores de "genres" y "playlists" estén vacías,
# lo cual significa que el usuario no quiere filtrar por género, y quiere seleccionar 
# todas las canciones de las playlists que sube. 

In [71]:
# Tomo los siguientes grupos como ejemplos:

mygroup1 = {"users": 
                   {"sedepa": {"playlists": ["MAD", "Correr", "A little of verguenza, please"],
                               "genres": ["spanish rock"]}, 
                    "zltm3cqxt0u0s9k4e8oy564dk": {"playlists": ["FPP"],
                                                  "genres": []}}, 
           "host": "zltm3cqxt0u0s9k4e8oy564dk",
           "new_playlist_name": "ÉXITO"}

In [72]:
# Defino una función que obtenga una lista con todas las canciones que hay en todas las 
# playlist que los usuarios me han pasado, filtrando por género en caso de que así lo
# hayan especificado.
# Para ello, primero tengo que obtener el ID correspondiente a cada nombre de playlist
# que me hayan pasado.

def get_tracks_info(group):
    tracks_info = []
    for user in group["users"].keys():
        
    # Si un usuario no seleccionan ninguna playlist, se cogen todas las de su perfil.
        if group["users"][user]["playlists"] == []:
            for playlist_ID in users_playlists_names_IDs[user].values():
                tracks_info.append(sp.user_playlist_tracks(user, playlist_id = playlist_ID))
        
        else:
            for playlist in group["users"][user]["playlists"]:
                tracks_info.append(sp.user_playlist_tracks(user, 
                                                           playlist_id = users_playlists_names_IDs[user][playlist]))
        
    return tracks_info

In [73]:
tracks_info = get_tracks_info(mygroup1)

In [74]:
len(tracks_info)

4

In [75]:
# Aquí he conseguido filtrar por género, utilizando la info del artista de cada canción.
# Tengo que meter este código generalizado (aquí está solo para sedepa) arriba en la función.

final_track_names = []
final_track_IDs = []

for playlist in tracks_info:
    for track in playlist["items"]:
        artist_id = track["track"]["album"]["artists"][0]["external_urls"]["spotify"][32:]
        artist_info = sp.artist(artist_id)
        genre_in_artist = 0
        if mygroup1["users"]["sedepa"]["genres"] == []:
            final_track_names.append(track["track"]["name"])
            final_track_IDs.append(track["track"]["id"])
        else:
            for genre in mygroup1["users"]["sedepa"]["genres"]:
                if genre in artist_info["genres"]:
                    genre_in_artist += 1
            if genre_in_artist > 0:
                final_track_names.append(track["track"]["name"])
                final_track_IDs.append(track["track"]["id"]) 

In [76]:
final_track_names

['A Ti Te Ocurre Algo',
 'Godzilla (feat. Enrique Bunbury & Ximena Sariñana)',
 'Como Si Fueras a Morir Mañana',
 'Expertos',
 'Nuclear',
 'No Te Preocupes por Mi',
 'Afuera en la Ciudad',
 'Guerra Mundial',
 'El Último Incendio',
 'Sincericidio',
 'Medicina',
 'Miedo',
 'Electricidad',
 'Ciencia Ficcion',
 'Breaking Bad',
 'A Ti Te Ocurre Algo',
 'Godzilla (feat. Enrique Bunbury & Ximena Sariñana)',
 'Como Si Fueras a Morir Mañana']

In [77]:
final_track_names = [item for items, c in Counter(final_track_names).most_common() for item in [items] * c]
final_track_IDs = [item for items, c in Counter(final_track_IDs).most_common() for item in [items] * c]

In [78]:
final_track_names

['A Ti Te Ocurre Algo',
 'A Ti Te Ocurre Algo',
 'Godzilla (feat. Enrique Bunbury & Ximena Sariñana)',
 'Godzilla (feat. Enrique Bunbury & Ximena Sariñana)',
 'Como Si Fueras a Morir Mañana',
 'Como Si Fueras a Morir Mañana',
 'Expertos',
 'Nuclear',
 'No Te Preocupes por Mi',
 'Afuera en la Ciudad',
 'Guerra Mundial',
 'El Último Incendio',
 'Sincericidio',
 'Medicina',
 'Miedo',
 'Electricidad',
 'Ciencia Ficcion',
 'Breaking Bad']

In [79]:
genres = ['deep house', 'minimal tech house', 'tropical house', 'spanish pop', 'spanish rock']